<a href="https://colab.research.google.com/github/poyashi/optuna/blob/master/20190523_vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from __future__ import print_function
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import BatchNormalization
import cv2
import os
import math
import numpy as np
import keras.callbacks
import keras.backend.tensorflow_backend as KTF
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

In [42]:
df5 = pd.read_csv(
    './drive/My Drive/DeepFashion/Category and Attribute Prediction Benchmark/Anno/list_category_img.txt',
    skiprows=1,
    sep=r'\s+')
df6 = pd.read_csv(
    './drive/My Drive/DeepFashion/Category and Attribute Prediction Benchmark/Anno/list_landmarks.txt',
    skiprows=1,
    sep=r'\s+')
df5_6 = pd.merge(df5, df6, on='image_name')
df5_6.head()

,image_name,category_label,clothes_type,variation_type,landmark_visibility_1,landmark_location_x_1,landmark_location_y_1,landmark_visibility_2,landmark_location_x_2,landmark_location_y_2,landmark_visibility_3,landmark_location_x_3,landmark_location_y_3,landmark_visibility_4,landmark_location_x_4,landmark_location_y_4,landmark_visibility_5,landmark_location_x_5,landmark_location_y_5,landmark_visibility_6,landmark_location_x_6,landmark_location_y_6,landmark_visibility_7,landmark_location_x_7,landmark_location_y_7,landmark_visibility_8,landmark_location_x_8,landmark_location_y_8
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,3,1,0,146,102,0,173,95,0,94,242,0,205,255,0.0,136.0,229.0,0.0,177.0,232.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,3,1,0,107,67,0,122,67,0,79,78,0,140,91,0.0,106.0,150.0,0.0,134.0,152.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,3,1,0,101,79,0,116,76,0,78,81,0,141,84,0.0,87.0,188.0,0.0,118.0,185.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,3,1,0,96,74,0,113,72,0,67,171,0,148,171,0.0,100.0,148.0,0.0,120.0,148.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,3,1,1,102,106,0,116,102,0,63,194,1,146,216,0.0,105.0,245.0,0.0,137.0,246.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
folders = os.listdir('./drive/My Drive/img_short')

In [0]:
h = 60
w = 40

In [45]:
X = []
Y = []
for i,folder in enumerate(folders):
  print(folder + " enter")
  names = os.listdir('./drive/My Drive/img_short/' + folder)
  for j,name in enumerate(names):
    im = cv2.imread('./drive/My Drive/img_short/' + folder + '/' + name)
    im2 = cv2.resize(im, (w, h))
    i = df5_6[df5_6.image_name == 'img/' + folder + '/' + name].clothes_type.values.tolist()
    X.append(im2)
    Y += i

2-in-1_Space_Dye_Athletic_Tank enter
25_Mesh-Paneled_Jersey_Dress enter
36_Plaid_Shirt_Dress enter
1981_Graphic_Ringer_Tee enter
Above_Average_Linen_Tee enter
Abstract_Print_Maxi_Skirt enter


In [0]:
X = np.array(X)
Y = np.array(Y)

In [47]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8, random_state=0)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

245 train samples
62 test samples


In [0]:
batch_size = 32
num_classes = 3
epochs = 10
data_augmentation = False

In [0]:
y_train = y_train-1
y_test = y_test-1

In [0]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
def model_family_cnn(input_shape, num_classes=3):
    model = Sequential()
    
    # Block 1
    model.add(Conv2D(64, (3, 3), input_shape=input_shape, name='conv1_1', activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3),name='conv1_2', activation='relu', padding='same'))
    model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    # Block 2
    model.add(Conv2D(128, (3, 3),name='conv2_1', activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3),name='conv2_2', activation='relu', padding='same'))
    model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    # Block 3
    model.add(Conv2D(256, (3, 3),name='conv3_1', activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3),name='conv3_2', activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3),name='conv3_3', activation='relu', padding='same'))
    model.add(Conv2D(256, (3, 3),name='conv3_4', activation='relu', padding='same'))
    model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    # Block 4
    model.add(Conv2D(512, (3, 3),name='conv4_1', activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3),name='conv4_2', activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3),name='conv4_3', activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3),name='conv4_4', activation='relu', padding='same'))
    model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    # Block 5
    model.add(Conv2D(512, (3, 3),name='conv5_1', activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3),name='conv5_2', activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3),name='conv5_3', activation='relu', padding='same'))
    model.add(Conv2D(512, (3, 3),name='conv5_4', activation='relu', padding='same'))
    model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))
    
    model.add(Flatten())
    model.add(Dense(4096))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model


In [0]:
input_shape = (h, w, 3)

In [0]:
old_session = KTF.get_session()

In [0]:
session = tf.Session('')
KTF.set_session(session)
KTF.set_learning_phase(1)


In [0]:
model = model_family_cnn(input_shape, num_classes = num_classes)

In [0]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [0]:
tb_cb = keras.callbacks.TensorBoard(log_dir="./tflog_vgg16/", histogram_freq=1)

In [0]:
cbks = [tb_cb]

In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_1 (Conv2D)             (None, 60, 40, 64)        1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 60, 40, 64)        36928     
_________________________________________________________________
batch_normalization_6 (Batch (None, 60, 40, 64)        256       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 30, 20, 64)        0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 30, 20, 64)        0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 30, 20, 128)       73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 30, 20, 128)       147584    
__________

In [0]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.
x_test /= 255.


In [62]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
#               callbacks=cbks,
              validation_split=0.1
             )
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
#                         callbacks=cbks,
                        validation_data=(x_test, y_test))


Not using data augmentation.
Instructions for updating:
Use tf.cast instead.
Train on 220 samples, validate on 25 samples
Epoch 1/10
220/220 [==============================] - 7s 31ms/step - loss: 2.0828 - acc: 0.3545 - val_loss: 1.5705 - val_acc: 0.4400
Epoch 2/10
220/220 [==============================] - 0s 2ms/step - loss: 1.5973 - acc: 0.3773 - val_loss: 1.4320 - val_acc: 0.5600
Epoch 3/10
220/220 [==============================] - 0s 2ms/step - loss: 1.5652 - acc: 0.3909 - val_loss: 1.3184 - val_acc: 0.3600
Epoch 4/10
220/220 [==============================] - 0s 2ms/step - loss: 1.4124 - acc: 0.4727 - val_loss: 1.1473 - val_acc: 0.6000
Epoch 5/10
220/220 [==============================] - 0s 2ms/step - loss: 1.4013 - acc: 0.4909 - val_loss: 1.3494 - val_acc: 0.4800
Epoch 6/10
220/220 [==============================] - 0s 2ms/step - loss: 1.2058 - acc: 0.5227 - val_loss: 1.3501 - val_acc: 0.4400
Epoch 7/10
220/220 [==============================] - 0s 2ms/step - loss: 1.3717 - ac

In [63]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 1.209571872988055
Test accuracy: 0.6129032238837211


In [0]:
KTF.set_session(old_session)